In [21]:
#bank management system
import random
import datetime
from abc import ABC,abstractmethod

class Insuffientbalance(Exception):
  pass
class Accountloacked(Exception):
  pass
class Invalidamount(Exception):
  pass


class Account:
  accountcount=0

  def __init__(self,name,initialdeposit=0):
    if  initialdeposit<0:
      raise Invalidamount("initial deposit cannot be negative")


    self.accountcount+=1

    self.name=name

    self.__balance=initialdeposit
    self._accountnumber=self._generateaccountnumber()
    self._isactive=True
    self._transhistroy=[]
    self._creationtime=datetime.datetime.now()

  def _generateaccountnumber(self):
    return f'Acc:{random.randint(10000000,99999999)}'
  def getaccountnumber(self):
    return self._accountnumber

  def checkaccountstatus(self):

      return self._isactive

  def checkbalance(self):
    return self.__balance

  def checkaccnumber(self):
    return self._accountnumber

  def addtranscation(self,type,amount,balanceafter=None):
    if balanceafter is None:
        balanceafter=self.__balance

    transcation=(f"TransType:{type} Amount:{amount} Balance:{balanceafter}")
    self._transhistroy.append(transcation)
    return

  def updatebalance(self,amount):
    self.__balance+=amount
    return

  def deposit(self,amount):
    self.updatebalance(amount)

    self.addtranscation("Deposit",amount)
    # Fix: self.updatebalance is a method, not the current balance
    print(f"Amount Deposited:{amount:.2f},Account balance is:{self.checkbalance():.2f}")

    return

  def withdraw(self,amount):
    self.updatebalance(-amount)
    # Fix: addtranscation expects two arguments, not a string minus an amount
    self.addtranscation("Withdraw",amount) # Assuming amount is positive here for logging
    # Fix: self.updatebalance is a method, not the current balance
    print(f"Amount Withdrawn:{amount:.2f},Account balance is:{self.checkbalance():.2f}")
    return

  def lockaccount(self):
    self._isactive=False
    return
  def unlockaccount(self):
    self._isactive=True
    return
  def gettranshistroy(self):
    # Fix: Correct attribute name and iteration for stored strings
    for transaction_str in self._transhistroy:
      print(transaction_str)
    return
  # Fix: Removed @abstractmethod as Account does not inherit from ABC.
  # Fix: Removed unreachable print statements.
  def getinfo(self):
    status="Active"if self._isactive else "Locked"
    return f"""

Account details ..
Account number:{self._accountnumber}.
Account status:{status}.
Account creation time:{self._creationtime}.
    """

class Savingsaccount(Account):
  def __init__(self,name,initaldeposit=0):
    super().__init__(name,initaldeposit)
    self._translimit=10
    self._interest=7.5
    self._transcount=0

  def checkintrest(self):
    return self._interest
  def applyintrest(self):
    interest=self.checkbalance()*(self.checkintrest()/100)
    self.deposit(interest)
    print(f"Interest applied:{interest:.2f}")
    transcation=(f"Interest:{interest:.2f}")
    self._transhistroy.append(transcation)
    return

  def withdraw(self,amount):
    if self._transcount<self._translimit:
      self.updatebalance(-amount)
      self.addtranscation("Withdraw",amount)
      self._transcount+=1
      print(f"Amount Withdrawn:{amount:.2f},Account balance is:{self.checkbalance():.2f}")
      return
    else:
      print(f"Transaction limit reached for today")
      return

  def gettype(self):
    status="Active"if self._isactive else "Locked"
    # Fix: intrest -> _interest
    return f"Saving account(intrest rate:{self._interest}),status:{status}."

  def resetwithdrawlimit(self):
    self._transcount=0
    return

class Currentaccount(Account):
  def __init__(self,name,initialdeposit=0,overdraft=1000):
    super().__init__(name,initialdeposit)
    self.updatebalance(overdraft) # This adds overdraft to balance, which might not be intended as initial state
    self._overdraft=overdraft
    self._translimit=20
    self._transfee=30
    self._transcount=0

  def checkoverdraft(self):
    return self._overdraft

  def withdraw(self,amount):
    if self._transcount<self._translimit:
      if self.checkbalance()+self._overdraft>=amount:
        self.updatebalance(-amount)
        self.addtranscation("Withdraw",amount)
        self._transcount+=1
        print(f"Amount Withdrawn:{amount:.2f},Account balance is:{self.checkbalance():.2f}")
        return
      # Fix: typo trancount -> _transcount, and translimit -> _translimit
      elif self._transcount>=self._translimit: # This condition might be intended for an overdraft fee
        self.updatebalance(-amount-self._transfee)
        self.addtranscation("Withdraw",amount)
        self._transcount+=1
        print(f"Amount Withdrawn:{amount:.2f},Account balance is:{self.checkbalance():.2f}")
        return
      elif amount>self.checkbalance(): # This else-if is already covered by the first if
        print(f"Insufficient balance and overdraft limit")
        return
    else:
        print(f"Transaction limit reached for today") # Added else block for translimit
    return


  def gettype(self):
    status="Active"if self._isactive else "Locked"
    # Fix: overdraft -> _overdraft
    return f"Current account(overdraft limit:{self._overdraft}),status:{status}"

  def resetwithdrawlimit(self):
    self._transcount=0
    return


class Bank:
  def __init__(self,name):
    self.name=name
    self.__accounts={}
    return

  def createaccount(self,name,accounttype,initialdeposit=0):
    account = None # Initialize account to None
    if accounttype=="savings":
      account=Savingsaccount(name,initialdeposit)
    elif accounttype=="current":
      account=Currentaccount(name,initialdeposit)

    if account: # Only add to accounts if an account was successfully created
      # Fix: Use account number as key, not method reference
      self.__accounts[account.getaccountnumber()]=account
    return account # Fix: Explicitly return the created account object

  def viewallaccounts(self):
    # Fix: Iterate over values (account objects) not keys
    for account_obj in self.__accounts.values():
      print(f"Account No:{account_obj.getaccountnumber()} Type:{account_obj.getinfo()}")

    return


def main():
  bank=Bank("Punjab National Bank")
  print(f"welcome to {bank.name}")

  account1=bank.createaccount("Harish","savings",10000)
  account2=bank.createaccount("Virgil","current",4000)

  if account1:
    print(f"Account 1 created: {account1.getaccountnumber()}")
  else:
    print("Account 1 creation failed.")

  if account2:
    print(f"Account 2 created: {account2.getaccountnumber()}")
  else:
    print("Account 2 creation failed.")

  # This line will now work because account1 is no longer None
  if account1: # Check again if account1 is not None before calling
    print(f"Harish's Account Number: {account1.getaccountnumber()}")
    account1.deposit(500)
    account1.withdraw(200)
    account1.applyintrest()
    account1.gettranshistroy()
    print(f"Harish's Account Info: {account1.getinfo()}")

  if account2:
    print(f"Virgil's Account Number: {account2.getaccountnumber()}")
    account2.withdraw(500)
    account2.withdraw(4000)
    account2.withdraw(1000) # Test overdraft
    account2.gettranshistroy()
    print(f"Virgil's Account Info: {account2.getinfo()}")

  bank.viewallaccounts()

if __name__=="__main__":
  main()


welcome to Punjab National Bank
Account 1 created: Acc:87459358
Account 2 created: Acc:65188048
Harish's Account Number: Acc:87459358
Amount Deposited:500.00,Account balance is:10500.00
Amount Withdrawn:200.00,Account balance is:10300.00
Amount Deposited:772.50,Account balance is:11072.50
Interest applied:772.50
TransType:Deposit Amount:500 Balance:10500
TransType:Withdraw Amount:200 Balance:10300
TransType:Deposit Amount:772.5 Balance:11072.5
Interest:772.50
Harish's Account Info: 

Account details ..
Account number:Acc:87459358.
Account status:Active.
Account creation time:2025-11-28 18:54:11.350669.
    
Virgil's Account Number: Acc:65188048
Amount Withdrawn:500.00,Account balance is:4500.00
Amount Withdrawn:4000.00,Account balance is:500.00
Amount Withdrawn:1000.00,Account balance is:-500.00
TransType:Withdraw Amount:500 Balance:4500
TransType:Withdraw Amount:4000 Balance:500
TransType:Withdraw Amount:1000 Balance:-500
Virgil's Account Info: 

Account details ..
Account number:Acc: